In [5]:
import re
import math
import warnings
import tqdm
import json
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize


warnings.filterwarnings("ignore")

In [6]:
data1 = pd.read_excel('2b数据处理结果.xlsx')
with open("best_labels.json", "r") as json_file:
    group_targets = json.load(json_file)
new_column = pd.Series(group_targets["best_labels"], name="组号")
data1 = pd.concat([data1, new_column], axis=1)

{'best_labels': 0.694639407240175}

In [7]:
data1

,ID,年龄,脑出血前mRS评分,高血压病史,卒中病史,糖尿病史,房颤史,冠心病史,吸烟史,饮酒史,血压,性别_男,性别_女,脑室引流,止血治疗,降颅压治疗,降压治疗,镇静、镇痛治疗,止吐护胃,营养神经
0,sub001,0,0.0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1
1,sub002,0,0.0,1,0,0,0,0,0,0,0,1,0,0,1,1,1,0,1,1
2,sub003,1,0.0,1,0,0,0,0,0,0,0,1,0,0,1,1,1,1,1,1
3,sub004,1,1.0,1,1,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0
4,sub005,0,0.0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,sub096,1,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1
96,sub097,1,0.0,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1
97,sub098,1,0.0,1,1,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1
98,sub099,0,0.0,1,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1


In [ ]:
# 按组号分组
grouped_data = data1.groupby("组号")
grouped_dataframes = {}
for group, group_df in grouped_data:
    grouped_dataframes[group] = group_df

In [ ]:
grouped_dataframes[group]

In [ ]:
# 模型融合

#线性
def liner_func(x,a,b,c,d):
    return a*x+b

#二次
def erchi_func(x,a,b,c,d):
    return a*x**2+b*x+c

#三次
def sanchi_func(x,a,b,c,d):
    return a*x**3+b*x**2+c*x+d

# 三角函数
def trig_func(x,a,b,c,d):
    return a*np.sin(x)+b*np.cos(x)+c
# 指数曲线
def target_func(x, a, b, c, d):
    return a * np.exp(-x / b) + c
# 对数函数
def hyp_func(x, a,b,c,d):
    return a*np.log(x)+b

def __sst(y_no_fitting):
    """
    计算SST(total sum of squares) 总平方和
    :param y_no_predicted: List[int] or array[int] 待拟合的y
    :return: 总平方和SST
    """
    y_mean = sum(y_no_fitting) / len(y_no_fitting)
    s_list =[(y - y_mean)**2 for y in y_no_fitting]
    sst = sum(s_list)
    return sst


def __ssr(y_fitting, y_no_fitting):
    """
    计算SSR(regression sum of squares) 回归平方和
    :param y_fitting: List[int] or array[int]  拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 回归平方和SSR
    """
    y_mean = sum(y_no_fitting) / len(y_no_fitting)
    s_list =[(y - y_mean)**2 for y in y_fitting]
    ssr = sum(s_list)
    return ssr


def __sse(y_fitting, y_no_fitting):
    """
    计算SSE(error sum of squares) 残差平方和
    :param y_fitting: List[int] or array[int] 拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 残差平方和SSE
    """
    s_list = [(y_fitting[i] - y_no_fitting[i])**2 for i in range(len(y_fitting))]
    sse = sum(s_list)
    return sse


def goodness_of_fit(y_fitting, y_no_fitting):
    """
    计算拟合优度R^2
    :param y_fitting: List[int] or array[int] 拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 拟合优度R^2
    """
    SSR = __ssr(y_fitting, y_no_fitting)
    SST = __sst(y_no_fitting)
    rr = SSR /SST
    return rr

def selected_func(you,model,model_select):
    maxyou=max(you)
    for s in range(len(you)):
        if you[s]==maxyou:
            
            return model[s],model_select[s]

def is_non_real(x):
    # 判断复数是否为非实数
    return math.isinf(x) or math.isnan(x)

def sgn(x):
    if x < 0:
        return -1
    elif x == 0:
        return 0
    else:
        return 1

# 求解方程 f(x) = y
def solve_equation(f,y):
    equation = lambda x: f(x) - y
    x_initial_guess = 0  # 初始猜测值
    x_solution = fsolve(equation, x_initial_guess)
    return x_solution

Set={}
youSet={}
resSet={}
resTime={}
for index, row in selected_df.iterrows():
    if index >= 100:
        break
    # 去除NaN值
    data = row.values[~np.isnan(row.values)]
    # 将数据分成x和y坐标对
    x = data[::2] / (3600 )
    y = data[1::2]/1000
    you=[]
    model=[]
    model_select=[]
    you_index={}
    try:
        a1 = np.polyfit(x, y, 1)#线性
        you1 = goodness_of_fit([liner_func(x[p],a1[0],a1[1]) for p in range(len(x))],y)
        you.append(you1)
        model.append({'回归类型':'线性回归','回归系数':a1})
        you_index['线性回归']=you1
        model_select.append((liner_func,a1+[0,0]))
    except:
        pass
    
    try:
        a2 = np.polyfit(x, y, 2)#二次
        you2 = goodness_of_fit([erchi_func(x[p],a2[0],a2[1],a2[2]) for p in range(len(x))],y)
        you.append(you2)
        model.append({'回归类型':'二次函数回归','回归系数':a2})
        you_index['二次函数回归']=you2
        model_select.append((erchi_func,a2+[0]))
    except:
        pass
    
    try:
        a3 = np.polyfit(x, y, 3)#三次
        you3 = goodness_of_fit([sanchi_func(x[p],a3[0],a3[1],a3[2],a3[3]) for p in range(len(x))],y)
        you.append(you3)
        model.append({'回归类型':'三次函数回归','回归系数':a3})
        you_index['三次函数回归']=you3
        model_select.append((sanchi_func,a3))
    except:
        pass
    
    #拟合三角函数模型
    try:
        a4,_=optimize.curve_fit(trig_func,x,y)
        you4 = goodness_of_fit([trig_func(x[p],a4[0],a4[1],a4[2]) for p in range(len(x))],y)
        you.append(you4)
        model.append({'回归类型':'三角函数回归','回归系数':a4})
        you_index['三角函数回归']=you4
        model_select.append((trig_func,a4+[0]))
        
    except:
        pass
    
    #拟合指数函数模型
    try:
        a5,_=optimize.curve_fit(target_func,x,y)
        you5 = goodness_of_fit([target_func(x[p], a5[0], a5[1], a5[2]) for p in range(len(x))],y)
        you.append(you5)
        model.append({'回归类型':'指数函数回归','回归系数':a5})
        you_index['指数函数回归']=you5
        model_select.append((target_func,a5+[0]))
    except:
        pass
    
    #拟合对数函数模型
    try:
        a6,_=optimize.curve_fit(hyp_func,x,y)
        you6 = goodness_of_fit([hyp_func(x[p], a6[0],a6[1]) for p in range(len(x))],y)
        you.append(you6)
        model.append({'回归类型':'对数函数回归','回归系数':a6})
        you_index['对数函数回归']=you6
        model_select.append((hyp_func,a6+[0,0]))
    except:
        pass
    
    # model=[f'线性回归，系数为{a1}',f'二次函数回归，系数为{a2}',f'三次函数回归，系数为{a3}',f'三角函数回归，系数为{a4}',f'指数函数回归，系数为{a5}',f'对数函数回归，系数为{a6}']
    res,model_=selected_func(you,model,model_select)
    
    Set[index]=res#选择函数模型，例如：1用三角函数
    youSet[index]=you_index#拟合优度
    best_position1, best_value1 = particle_swarm_optimization(1,model_[0], 50, 500,model_[1])
    # best_value1=round(float(best_value1),4)
    # print("最小值:",best_value1,'最小位置:',best_position1)
    best_position2, best_value2 = particle_swarm_optimization(-1,model_[0], 50, 500,model_[1])

    # best_value2=round(float(-best_value2),4)
    # print("最大值:", -best_value2,'最大位置:',best_position2)
    resSet[index+1]=(y[0],float(-best_value2),x[0],best_position2)
    x1 = symbols('x')
    def y1(x):
        return model_[0](x,*model_[1])
    if float(best_value1)<0:
        best_value1=0.1
    
    # 在第一次检测的的右边的非复数解
    
    solution11 = solve_equation(y1,float(y[0])+6)
    solution1=[]
    for u in solution11:
        if str(u)[-1]!='I' and u>=x[0]:
            solution1.append(u)
       
    solution22 = solve_equation(y1,float(y[0])*1.33)
    solution2=[]
    for u in solution22:
        if str(u)[-1]!='I' and u>=x[0]:
            solution2.append(u)

    if solution1 == [] and solution2 == []:
        resTime[index+1]=-999
    elif solution1 == [] and solution2 != []:
        if solution2[0] <= 48:
            resTime[index+1]=round(solution2[0],4)
        else:
            resTime[index+1]=-999
    elif solution2 == [] and solution1 != []:
        if solution1[0] <= 48:
            resTime[index+1]=round(solution1[0],4)
        else:
            resTime[index+1]=-999
    else:
        if solution1[0] > 48 and solution2[0] > 48:
            resTime[index+1]=-999
        else:
            resTime[index+1]=round(min(solution1[0],solution2[0]),4)
            
    

dicts1 = [{key: value for key, value in Set[row].items()} for row in Set]
df1 = pd.DataFrame(dicts1)
df1.to_excel('血肿2b_160人拟合函数选择结果.xlsx')
dicts2 = [{key: value for key, value in youSet[row].items()} for row in youSet]
df2 = pd.DataFrame(dicts2)
df2.to_excel('血肿2b_160人拟合优度.xlsx')